# Activity recognition on the Capture24 dataset -- Convolutional neural networks

*This section assumes familiarity with [PyTorch](https://pytorch.org/)*

In this section, instead of using the hand-crafted features, we use a neural network on the raw accelerometer measurements so that we let the neural network automatically learn relevant features for classification.

Setup:

In [ ]:
import numpy as np
from scipy.ndimage import median_filter
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)
cudnn.benchmark = True

 Load dataset and hold out some instances for testing: 

In [ ]:
# X_raw = np.memmap('X_raw.dat', dtype='float32', mode='r').reshape(-1,3,3000)
X_raw = np.load('X_raw_small.npy')
print("Raw data shape:", X_raw.shape)
# data = np.load('capture24.npz', allow_pickle=True)
data = np.load('capture24_small.npz', allow_pickle=True)
print("Data contents:", data.files)
y, pid, time = data['y'], data['pid'], data['time']

# Hold out some participants for testing the model
test_pids = [2, 3]
test_mask = np.isin(pid, test_pids)
train_mask = ~np.isin(pid, test_pids)
X_train, y_train, pid_train = X_raw[train_mask], y[train_mask], pid[train_mask]
X_test, y_test, pid_test = X_raw[test_mask], y[test_mask], pid[test_mask]
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

A standard procedure is to normalize the input data, which aids the optimization of neural networks:

In [ ]:
scaler = preprocessing.StandardScaler(copy=False)
X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0],-1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(X_test.shape[0],-1)).reshape(X_test.shape)

We start by defining a simple feed-forward convolutional neural network (CNN):

In [ ]:
class CNN(nn.Module):
    def __init__(self, output_dim, in_channels, num_filters_base=8):
        super(CNN, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels, num_filters_base,
                8, 4, 2, bias=False),  # state shape (N,num_filters_base,750)
            nn.BatchNorm1d(num_filters_base),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base, num_filters_base*2,
                6, 4, 2, bias=False),  # state shape (N,num_filters_base*2,188)
            nn.BatchNorm1d(num_filters_base*2),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*2, num_filters_base*4,
                8, 4, 2, bias=False),  # state shape (N,num_filters_base*4,47)
            nn.BatchNorm1d(num_filters_base*4),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*4, num_filters_base*8,
                3, 2, 1, bias=False),  # state shape (N,num_filters_base*8,24)
            nn.BatchNorm1d(num_filters_base*8),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*8, num_filters_base*16,
                4, 2, 1, bias=False),  # state shape (N,num_filters_base*16,12)
            nn.BatchNorm1d(num_filters_base*16),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*16, num_filters_base*32,
                4, 2, 1, bias=False),  # state shape (N,num_filters_base*32,6)
            nn.BatchNorm1d(num_filters_base*32),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*32, num_filters_base*64,
                6, 1, 0, bias=False),  # state shape (N,num_filters_base*64,1)
            nn.BatchNorm1d(num_filters_base*64),
            nn.ReLU(True),
            nn.Conv1d(num_filters_base*64, output_dim,
                1, 1, 0, bias=True)  # state shape (N,output_dim,1)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm1d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.cnn(x).squeeze()

 If there is a GPU, let's use it! 

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    # torch.cuda.set_device(args.device)
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

 Define a dataset iterator that will provide the mini-batches during the training, instantiate the neural network, and define the loss function and optimizer: 

In [ ]:
max_iter = 10
lr = 1e-4
num_filters_base = 8
batch_size = 32

X_train_torch = torch.from_numpy(X_train.astype('float32')).to(device)
y_train_torch = torch.from_numpy(y_train).to(device)
def create_dataloader(batch_size=1):
    nbatch = X_train_torch.shape[0] // batch_size
    idxs = np.random.permutation(np.arange(X_train_torch.shape[0]))
    for i in range(nbatch):
        _idxs = idxs[i:i+batch_size]
        yield X_train_torch[_idxs], y_train_torch[_idxs]

cnn = CNN(
    output_dim=utils.NUM_CLASSES,
    in_channels=X_train_torch.shape[1],
    num_filters_base=num_filters_base
).to(device)
print(cnn)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr, amsgrad=True)

 Training via mini-batch gradient descent begins here. We loop through the training set `max_iter` times with the `dataloader` iterator.

In [ ]:
losses = []
for i in tqdm(range(max_iter)):
    dataloader = create_dataloader(batch_size)
    for x, y in dataloader:
        cnn.zero_grad()
        output = cnn(x)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

# View loss during training
utils.plt.semilogy(median_filter(losses, size=100))
utils.plt.savefig('cnn_loss.png')

 Evaluate the model on the hold out set 

In [ ]:
cnn.eval()
with torch.no_grad():
    output = cnn(torch.from_numpy(X_test.astype('float32')).to(device))
    y_test_pred = torch.argmax(F.softmax(output, dim=1), dim=1).cpu().numpy()
print("Cohen kappa score:", utils.cohen_kappa_score(y_test, y_test_pred, pid_test))
print("Accuracy score:", utils.accuracy_score(y_test, y_test_pred, pid_test))

#TODO: HMM smoothing
